In [ ]:
# ITERATIVE BACKTEST

## Compare with short selling / Entering rf when not in asset
## Make Modular with functions

In [ ]:
# Importing Libraries
import numpy as np
import pandas as pd
import datetime as dt
import yfinance as yf
from pandas_datareader import data as pdr

In [ ]:
# Variables
end = dt.datetime(2024, 1, 1)
start = dt.datetime(2023, 1, 1)
stock = ["GOOGL"]

# Data
df = yf.download(stock, start, end)
df = df.iloc[:, [0, 3, 4]]

# RSI and Return Calc
df["Returns"] = (df[("Adj Close")] / df[("Adj Close")].shift(1)) - 1

def RSICalc(x: list) -> float:
    avg_gain = x[x > 0].mean()
    avg_loss = -1 * x[x < 0].mean()
    RS = avg_gain / avg_loss
    RSI = 100 - (100 / (1 + RS))
    return RSI

df["RSI"] = df["Returns"].rolling(14).apply(RSICalc)
df = df.dropna()

# Signal Generation
df["Signal"] = 0
i = 0

for i in range(len(df["Signal"])):

    if df.iloc[i, 4] < 30:
        df.iloc[i, 5] = 1

    if 35 < df.iloc[i, 4] < 75:
        if df.iloc[i - 1, 5] == 1:
            df.iloc[i, 5] = 1
        
    if df.iloc[i, 4] > 70:
        df.iloc[i, 5] = 0



In [ ]:

# Backtest
df["Cash"] = 1000

for i in range(len(df["Cash"])):
    
    if df.iloc[i - 1, 5] == 1:
        df.iloc[i, 6] = df.iloc[i - 1, 6] * (1 + df.iloc[i, 3])
        
    elif df.iloc[i - 1, 5] == 0:
        df.iloc[i, 6] = df.iloc[i - 1, 6]

# Trading Costs
TC = 0.01
df["Trading_Costs"] = 0

for i in range(len(df["Trading_Costs"])):

    if df.iloc[i, 5] == 0 and df.iloc[i - 1, 5] == 1:
            df.iloc[i, 7] = df.iloc[i, 6] * TC

    elif df.iloc[i, 5] == 1 and df.iloc[i - 1, 5] == 0:
         df.iloc[i, 7] = df.iloc[i, 6] * TC

    elif df.iloc[i, 5] == 0 and df.iloc[i -1, 5] == 0:
          df.iloc[i, 7] = 0

# Profit Calc
Total_TC = df["Trading_Costs"].sum()
Profit = df.iloc[-1, 6] - (1000 + Total_TC)

# Returns Calc
df["Strat_Returns"] = (df[("Cash")] / df[("Cash")].shift(1)) - 1
df = df.fillna(0)
Strat_Return = ((df.iloc[-1, 6] / df.iloc[0, 6]) - 1) * 100
TC_Adj_Return = (((df.iloc[-1, 6] - Total_TC) / df.iloc[0, 6]) - 1) * 100
Buy_Hold_Return = ((df.iloc[-1, 2] / df.iloc[0, 2]) - 1) * 100

# Sharpe Ratio Calc
def Sharpe(return_data: list, RF: float, Unit_Time: int) -> float:

    Num = return_data.mean() * Unit_Time - RF
    Denom = return_data.std() * np.sqrt(Unit_Time)

    return Num / Denom

Strat_Sharpe = Sharpe(df["Strat_Returns"], .045, 252)
Buy_Hold_Sharpe = Sharpe(df["Returns"], .045, 252)

# Printing Results
print(f"Profit = ${Profit:.2f}")
print(f"Strategy Return = {Strat_Return:.2f}%")
print(f"TC Adj Strategy Return = {TC_Adj_Return:.2f}%")
print(f"Strategy Sharpe Ratio = {Strat_Sharpe:.2f}")
print(f"Buy and Hold Return = {Buy_Hold_Return:.2f}%")
print(f"Buy and Hold Sharpe Ratio = {Buy_Hold_Sharpe:.2f}")
